## Introduction to Keras callbacks
In Keras, Callback is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can be called at different stages of training/evaluating/inference. Keras has available callbacks and we'll show how you can use it in the following sections. Please click the Open in Colab badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following tf.keras.Model methods:

- fit(), fit_generator() Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
- evaluate(), evaluate_generator() Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
- predict(), predict_generator() Generates output predictions for the input data or data generator.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.9.0-dev20220315


## Examples of Keras callback applications
The following section will guide you through creating simple Callback applications.

In [2]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## TensorBoard

Enable visualizations for TensorBoard.

In [9]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 10s 340ms/step - loss: 0.6712 - accuracy: 0.5925 - val_loss: 0.6407 - val_accuracy: 0.6976
Epoch 2/10
26/26 [==============================] - 8s 324ms/step - loss: 0.6032 - accuracy: 0.6898 - val_loss: 0.6014 - val_accuracy: 0.6244
Epoch 3/10
26/26 [==============================] - 8s 315ms/step - loss: 0.5515 - accuracy: 0.7275 - val_loss: 0.5149 - val_accuracy: 0.7610
Epoch 4/10
26/26 [==============================] - 10s 370ms/step - loss: 0.5145 - accuracy: 0.7652 - val_loss: 0.4621 - val_accuracy: 0.7561
Epoch 5/10
26/26 [==============================] - 9s 336ms/step - loss: 0.4804 - accuracy: 0.7749 - val_loss: 0.4616 - val_accuracy: 0.8195
Epoch 6/10
26/26 [==============================] - 9s 336ms/step - loss: 0.3994 - accuracy: 0.8321 - val_loss: 0.3741 - val_accuracy: 0.8049
Epoch 7/10
26/26 [==============================] - 9s 334ms/step - loss: 0.3762 - accuracy: 0.8491 - val_loss: 0.3272 - val_accuracy: 0.8146
Epoc

In [10]:
%tensorboard --logdir logs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2022-04-01 08:07:32.678335: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:07:36.784358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:07:36.785290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-04-01 08:07:36.786220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-04-01 08:07:36.787257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cufft64_10.dll'; dlerror: cufft64_10.dll not found
2022-04-01 08:07:36.788122: W tensor

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 1: saving model to saved_model


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


26/26 - 10s - loss: 0.6752 - accuracy: 0.5584 - val_loss: 0.6785 - val_accuracy: 0.4341 - 10s/epoch - 366ms/step


In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 1: saving model to model.h5
26/26 - 9s - loss: 0.6674 - accuracy: 0.6083 - val_loss: 0.6679 - val_accuracy: 0.5024 - 9s/epoch - 357ms/step
Epoch 2/2

Epoch 2: saving model to model.h5
26/26 - 8s - loss: 0.6247 - accuracy: 0.6740 - val_loss: 0.5852 - val_accuracy: 0.7756 - 8s/epoch - 316ms/step


## Early stopping
Stop training when a monitored metric has stopped improving.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 8s - loss: 0.6686 - accuracy: 0.5961 - val_loss: 0.6443 - val_accuracy: 0.6488 - 8s/epoch - 323ms/step
Epoch 2/50
26/26 - 8s - loss: 0.6142 - accuracy: 0.7092 - val_loss: 0.5965 - val_accuracy: 0.7073 - 8s/epoch - 308ms/step
Epoch 3/50
26/26 - 8s - loss: 0.5574 - accuracy: 0.7384 - val_loss: 0.5535 - val_accuracy: 0.7366 - 8s/epoch - 315ms/step
Epoch 4/50
26/26 - 10s - loss: 0.5125 - accuracy: 0.7555 - val_loss: 0.4992 - val_accuracy: 0.7805 - 10s/epoch - 369ms/step
Epoch 5/50
26/26 - 8s - loss: 0.4396 - accuracy: 0.8273 - val_loss: 0.4575 - val_accuracy: 0.8098 - 8s/epoch - 312ms/step
Epoch 6/50
26/26 - 8s - loss: 0.4073 - accuracy: 0.8236 - val_loss: 0.4535 - val_accuracy: 0.8098 - 8s/epoch - 304ms/step
Epoch 7/50
Restoring model weights from the end of the best epoch: 4.
26/26 - 8s - loss: 0.3189 - accuracy: 0.8929 - val_loss: 0.7979 - val_accuracy: 0.6098 - 8s/epoch - 304ms/step
Epoch 7: early stopping


## CSV Logger
Callback that streams epoch results to a CSV file.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 8s 315ms/step - loss: 0.6676 - accuracy: 0.5912 - val_loss: 0.6387 - val_accuracy: 0.7756
Epoch 2/5
26/26 [==============================] - 8s 305ms/step - loss: 0.6198 - accuracy: 0.6800 - val_loss: 0.6462 - val_accuracy: 0.5073
Epoch 3/5
26/26 [==============================] - 8s 314ms/step - loss: 0.5603 - accuracy: 0.7518 - val_loss: 0.5735 - val_accuracy: 0.6098
Epoch 4/5
26/26 [==============================] - 8s 315ms/step - loss: 0.4956 - accuracy: 0.7871 - val_loss: 0.6885 - val_accuracy: 0.5073
Epoch 5/5
26/26 [==============================] - 9s 341ms/step - loss: 0.4279 - accuracy: 0.8370 - val_loss: 0.3582 - val_accuracy: 0.9122


In [15]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.591241,0.667625,0.775610,0.638670
1,1,0.680049,0.619822,0.507317,0.646219
2,2,0.751825,0.560286,0.609756,0.573475
3,3,0.787105,0.495628,0.507317,0.688501
4,4,0.836983,0.427870,0.912195,0.358151


## Learning Rate Scheduler
Updates the learning rate during training.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 8s 313ms/step - loss: 0.6778 - accuracy: 0.5608 - val_loss: 0.6651 - val_accuracy: 0.6537 - lr: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 9s 337ms/step - loss: 0.6531 - accuracy: 0.6436 - val_loss: 0.6633 - val_accuracy: 0.5366 - lr: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 9s 336ms/step - loss: 0.6419 - accuracy: 0.6642 - val_loss: 0.6521 - val_accuracy: 0.6146 - lr: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 9s 335ms/step - loss: 0.6352 - accuracy: 0.7032 - val_loss: 0.6507 - val_accuracy: 0.6146 - lr: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] -

In [17]:
%tensorboard --logdir log_dir

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2022-04-01 08:10:31.351317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:10:35.165398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:10:35.166315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-04-01 08:10:35.167200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-04-01 08:10:35.174566: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cufft64_10.dll'; dlerror: cufft64_10.dll not found
2022-04-01 08:10:35.175444: W tensor

## ReduceLROnPlateau
Reduce learning rate when a metric has stopped improving.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 9s 317ms/step - loss: 0.6731 - accuracy: 0.5779 - val_loss: 0.6720 - val_accuracy: 0.4780 - lr: 0.0100
Epoch 2/50
26/26 [==============================] - 8s 322ms/step - loss: 0.6398 - accuracy: 0.6679 - val_loss: 0.6205 - val_accuracy: 0.8098 - lr: 0.0100
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.5803 - accuracy: 0.7518
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 8s 316ms/step - loss: 0.5803 - accuracy: 0.7518 - val_loss: 0.7141 - val_accuracy: 0.4683 - lr: 0.0100
Epoch 4/50
26/26 [==============================] - 8s 316ms/step - loss: 0.5360 - accuracy: 0.7871 - val_loss: 0.5316 - val_accuracy: 0.8000 - lr: 0.0020
Epoch 5/50
26/26 [==============================] - 8s 298ms/step - loss: 0.5107 - accuracy: 0.8163 - val_loss: 0.5238 - val_accuracy: 0.8195 - lr: 0.0020
Epoch 6/50
26/26 [==============================] - 8s 326ms/step 

In [19]:
%tensorboard --logdir log_dir

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2022-04-01 08:17:31.378512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:17:35.109510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-01 08:17:35.110425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-04-01 08:17:35.111318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-04-01 08:17:35.115408: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cufft64_10.dll'; dlerror: cufft64_10.dll not found
2022-04-01 08:17:35.116288: W tensor